In [7]:
import numpy as np
import pandas as pd

def sigmoid(x,w):
  arg = np.dot(x,w.T)
  return 1/(1 + np.exp(-arg))

def pred(y):
  return (np.argmax(y, axis=1) + 1)

def holdout(data, trainp, validp):
  m = np.shape(data)[0]

  train = data[0 : int(np.floor(m*trainp/100))]
  valid = data[int(np.floor(m*trainp/100)) : int(np.floor(m*trainp/100))+int(np.floor(m*validp/100))]
  test = data[int(np.floor(m*trainp/100))+int(np.floor(m*validp/100)) : None]

  y_train = train[:,-1] #shape = (rows,)
  x_train = np.delete(train, -1, axis=1)

  y_valid = valid[:,-1] #shape = (rows,)
  x_valid = np.delete(valid, -1, axis=1)

  y_test = test[:,-1] #shape = (rows,)
  x_test = np.delete(test, -1, axis=1)

  return x_train,x_valid,x_test,y_train,y_valid,y_test

def appendones(x):
  m = np.shape(x)[0]
  return np.concatenate((np.ones((m,1)),x), axis=1)

def one_hot_encode(y):
  len = np.size(y)
  encoded_y = np.zeros((len,3))
  for i in range(len):
    if y[i] == 1:
      encoded_y[i,0] = 1
    if y[i] == 2:
      encoded_y[i,1] = 1
    if y[i] == 3:
      encoded_y[i,2] = 1
  return encoded_y

def decode(y):
  len = np.shape(y)[0]
  class_labels = np.zeros(len)
  for i in range(len):
    class_labels[i] = np.argmax(y[i])+1
  return class_labels

def performance(y, pred):
  m = np.zeros((3,3)) # confusion matrix
  for p in range(len(pred)):
    if pred[p]==1 and y[p]==1:
      m[0,0]+=1
    if pred[p]==2 and y[p]==2:
      m[1,1]+=1
    if pred[p]==3 and y[p]==3:
      m[2,2]+=1
    if pred[p]==1 and y[p]==2:
      m[1,0]+=1
    if pred[p]==1 and y[p]==3:
      m[2,0]+=1
    if pred[p]==2 and y[p]==1:
      m[0,1]+=1
    if pred[p]==2 and y[p]==3:
      m[2,1]+=1
    if pred[p]==3 and y[p]==1:
      m[0,2]+=1
    if pred[p]==3 and y[p]==2:
      m[1,2]+=1
  ind_accuracy = [m[0,0]/np.sum(m[0,:]), m[1,1]/np.sum(m[1,:]), m[2,2]/np.sum(m[2,:])]
  accuracy = (m[0,0]+m[1,1]+m[2,2])/np.sum(m)
  return ind_accuracy, accuracy

def ELM_auto_encoder(x):
  v = np.random.randn(x.shape[1]-1,x.shape[1]) # random weights
  H = sigmoid(x,v)
  w = np.linalg.pinv(H) @ x # linear parameter solving with target as input  
  return v,H

In [8]:
data = pd.read_excel('/content/drive/MyDrive/NNFL Assignments (Aug 2021)/Assignment 2/data5.xlsx')
cols = np.array(data.columns, ndmin=2)
data = data.to_numpy()
data = np.concatenate((cols,data), axis=0)

# shuffle & train-test-valid-split
np.random.seed(0)
np.random.shuffle(data)
x_train, x_valid, x_test, y_train, y_valid, y_test = holdout(data, 70, 10)

# normalizing input data
mu = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)

x_train = (x_train-mu)/std
x_valid = (x_valid-mu)/std
x_test = (x_test-mu)/std

# one-hot-encoding output data
y_train_coded = one_hot_encode(y_train)
y_valid_coded = one_hot_encode(y_valid)
y_test_coded = one_hot_encode(y_test)

# appending ones (input for bias)
x_train = appendones(x_train)
x_valid = appendones(x_valid)
x_test = appendones(x_test)

m_train = np.size(y_train)
m_valid = np.size(y_valid)
m_test = np.size(y_test)

In [12]:
#### TRAINING ####
# layer 1
np.random.seed(0)
v1,H1 = ELM_auto_encoder(x_train)

# layer 2
np.random.seed(1)
v2,H2 = ELM_auto_encoder(H1)

# layer 3
L = 15
np.random.seed(3)
v3 = np.random.randn(L,H2.shape[1])
H3 = sigmoid(H2,v3)
w3 = w = np.linalg.pinv(H3) @ y_train_coded

#### TESTING ####
H1 = sigmoid(x_test,v1)
H2 = sigmoid(H1,v2)
H3 = sigmoid(H2,v3)
y = H3 @ w3
pred = decode(y)

# performance measures
ind_accuracy, accuracy = performance(y_test, pred)
print("accuracy of class 1 = {}".format(ind_accuracy[0]))
print("accuracy of class 2 = {}".format(ind_accuracy[1]))
print("accuracy of class 3 = {}".format(ind_accuracy[2]))
print("overall accuracy of classifier = {}".format(accuracy))

accuracy of class 1 = 0.9
accuracy of class 2 = 1.0
accuracy of class 3 = 0.9473684210526315
overall accuracy of classifier = 0.9523809523809523
